In [1]:
import pandas as pd
import os
from glob import glob
import numpy as np
from pandas.api.types import CategoricalDtype
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [2]:
ex1 = pd.read_parquet('회원_신용_청구.parquet')

In [10]:
X = ex1.drop(columns=['ID', '기준년월', 'Segment'])
X = X.select_dtypes(include=[np.number])
y = pd.factorize(ex1['Segment'], sort=True)[0]  # A~E → 0~4로 인코딩

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.4,
    stratify=y,
    random_state=42
)

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=11,
    class_weight='balanced',  # ← 클래스 불균형 보정
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print("★ Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))

test_df = pd.read_parquet("회원_신용_청구_test.parquet")

X_submit = test_df.drop(columns=['ID', '기준년월'])
X_submit = X_submit.select_dtypes(include=[np.number])

y_submit_pred = rf.predict(X_submit)

segment_labels = pd.factorize(ex1['Segment'], sort=True)[1]  # Index([A, B, C, D, E], dtype=object)
predicted_segments = segment_labels[y_submit_pred]

predict = pd.DataFrame({
    'ID': test_df['ID'],
    'Segment': predicted_segments
})

print(predict.head())

predict.to_csv("rf_model.csv", index=False)

MemoryError: Unable to allocate 897. MiB for an array with shape (49, 2400000) and data type int64

In [8]:
df_pred = pd.read_csv("rf_model.csv")

sample = pd.read_csv("sample_submission.csv")
print("샘플 행 수:", len(sample))  # 100,000

df_mode = df_pred.groupby("ID")["Segment"].agg(lambda x: x.mode().iloc[0]).reset_index()

final_submission = sample[['ID']].merge(df_mode, on='ID', how='left')

final_submission['Segment'] = final_submission['Segment'].fillna('A')

final_submission.to_csv("rf_final.csv", index=False)


샘플 행 수: 100000
